# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

# Reading Data

In [3]:
from google.colab import files
uploaded = files.upload()

Saving shahnameh.txt to shahnameh.txt


In [4]:
text = open('shahnameh.txt','r').read()

In [5]:
#text = text.replace("|", "")

In [6]:
print(text[:500])

|به نام خداوند جان و خرد
|کزین برتر اندیشه برنگذرد
|خداوند نام و خداوند جای
|خداوند روزی ده رهنمای
|خداوند کیوان و گردان سپهر
|فروزنده ماه و ناهید و مهر
|ز نام و نشان و گمان برترست
|نگارندهٔ بر شده پیکرست
|به بینندگان آفریننده را
|نبینی مرنجان دو بیننده را
|نیابد بدو نیز اندیشه راه
|که او برتر از نام و از جایگاه
|سخن هر چه زین گوهران بگذرد
|نیابد بدو راه جان و خرد
|خرد گر سخن برگزیند همی
|همان را گزیند که بیند همی
|ستودن نداند کس او را چو هست
|میان بندگی را ببایدت بست
|خرد را و جان را همی سنجد ا


# Preprocessing Data
 - removing unnecessary chars
 - encoding chars

In [7]:
vocab = sorted(set(text))

In [8]:
len(vocab)

48

In [9]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}

In [10]:
ind_to_char = np.array(vocab)

In [11]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [12]:
seq_len = 24

In [13]:
total_num_seq = len(text) // (seq_len + 1)

In [14]:
total_num_seq

106153

# Creating sequences

In [15]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [16]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [17]:
def create_seq_target(seq):
    input_text = seq[:-1]
    target_text = seq[1:]
    return input_text,target_text

In [18]:
dataset = sequences.map(create_seq_target)

In [19]:
for input_text,target_text in dataset.take(1):
    print(input_text.numpy())
    print("".join(ind_to_char[input_text.numpy()]))
    print('\n')
    print(target_text.numpy())
    print("".join(ind_to_char[target_text.numpy()]))

[ 4 15 38  1 37 14 36  1 20 21 14 39 37 21  1 18 14 37  1 39  1 20 23 21]
|به نام خداوند جان و خرد


[15 38  1 37 14 36  1 20 21 14 39 37 21  1 18 14 37  1 39  1 20 23 21  0]
به نام خداوند جان و خرد



In [20]:
batch_size = 128

In [21]:
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

# Model

In [22]:
vocab_size = len(vocab)

In [23]:
embed_dim = 64

In [24]:
rnn_neurons = 1026

In [25]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [26]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GRU,Embedding

In [28]:
# function to create model : we need to create model for forecasting with one sample
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    with tf.device('/gpu:0'):
        model = Sequential()
        model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
        model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
        model.add(Dense(vocab_size))
        model.compile(optimizer='adam',loss=sparse_cat_loss)
        return model

In [29]:
model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size)

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 64)           3072      
                                                                 
 gru (GRU)                   (128, None, 1026)         3361176   
                                                                 
 dense (Dense)               (128, None, 48)           49296     
                                                                 
Total params: 3413544 (13.02 MB)
Trainable params: 3413544 (13.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
for input_example_batch,target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)

# Training

In [32]:
epochs = 30

In [33]:
with tf.device('/gpu:0'):
    model.fit(dataset,epochs=epochs)

Epoch 1/30
829/829 [==============================] - 26s 28ms/step - loss: 2.0639
Epoch 2/30
829/829 [==============================] - 23s 27ms/step - loss: 1.5963
Epoch 3/30
829/829 [==============================] - 23s 28ms/step - loss: 1.4995
Epoch 4/30
829/829 [==============================] - 24s 29ms/step - loss: 1.4518
Epoch 5/30
829/829 [==============================] - 24s 28ms/step - loss: 1.4202
Epoch 6/30
829/829 [==============================] - 23s 28ms/step - loss: 1.3969
Epoch 7/30
829/829 [==============================] - 24s 28ms/step - loss: 1.3771
Epoch 8/30
829/829 [==============================] - 26s 30ms/step - loss: 1.3602
Epoch 9/30
829/829 [==============================] - 24s 28ms/step - loss: 1.3473
Epoch 10/30
829/829 [==============================] - 24s 28ms/step - loss: 1.3362
Epoch 11/30
829/829 [==============================] - 24s 28ms/step - loss: 1.3271
Epoch 12/30
829/829 [==============================] - 24s 28ms/step - loss: 1.3191
E

# Saving the model

In [34]:
model.save('shahname.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Creating new model with the same weights with batchsize 1

In [35]:
model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)

In [36]:
model.load_weights('shahname.h5')
model.build(input_shape=tf.TensorShape([1,None]))



### function for generating  Text


In [37]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):

    num_generate = gen_size
    input_eval = [char_to_ind[s] for s in start_seed]
    input_eval = tf.expand_dims(input_eval,0)
    text_generated = []

    temperature = temp

    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions,0)

        predictions = predictions / temperature

        predicted_id = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id],0)

        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [38]:
print(generate_text(model,'ز گفتار تا عود و عود و عبیر',gen_size=1000))

ز گفتار تا عود و عود و عبیر
|چو کو تیره شد ناپدید
|دل و میختن نشینمش کسری خردمند بیش
|زترک برادرت روزگار
|سزد گرتنک گاه و ببخت اندر آرم به خسرو رسید
|بدانجا خوب‌دلمش کند
|بدانست کامد ز گفتار سام ازو بازگشت
|یرسر بدندی به شاه
|چنین گفت کای بخرهٔ مست مس
|نیاید دل شهریار
|چوباکاید بباد
|بدان تا نبد داور کرد خاقان که نشنیده باشیدم این مرو
|گروم اندر آورد گیلان رسید
|به پیوستگاه
|فرخ‌فراز
|ز هر مهتری جاودان نامه برخاستند
|در و دشت ارمنیها در نگاور نکوسد پاسخ کم که لشکر به گفتار سوی خدنگ
|ن و دل باش و پاکان آوردگه کاستست
|جهانجوی ورشور تنبند اندر شگفت
|که گه کارست زین خویشتن رابم خسرو آراستند
|سوی کاخ بگرفت نیزه برخاستخواند
|وگر ان بپذر کلاه ویست
|نیازد به نزش اندرون بسته و بگذرد
|تو خشم
|کنون دیگها برگرفتند و تیر
|چو سالار بابک و پر از باد دارا منم
|بیارند آن ماه روی
|چو اسفندیار
|ش به مرا روزگار نبرد
|چو آشفته‌تخ بود
|ز کار زهارش شود تفت
|پیشه پای کودک سخنها که نیکیست شیر
|کمر بر سم کار
|چو بر زد کزویست بی نامدار افسرت
|ببد بی‌سپاهم چو سرو
|نماند مشان این بدان سنگ ماند ایستاده به هر گوژ پا